In [4]:
import tensorflow as tf
import cv2
import os
import numpy as np
os.environ['TF_CPP_MIN_LEVEL'] = '2'  # to avoid absurd messages in terminal
os.environ['KMP_DUPLICATE_LIB_OK']='True'

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

from tensorflow import keras
from tensorflow.keras import layers, regularizers

from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [5]:
# create a data generator
datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, horizontal_flip=True, rotation_range=5, )


# load and iterate training dataset
train_it = datagen.flow_from_directory('DataSet/train/', class_mode='categorical', batch_size=16,target_size=(224,224), subset = 'training')


# load and iterate test dataset
test_it = datagen.flow_from_directory('DataSet/test/', class_mode='categorical', batch_size=16 , target_size=(224,224), )


Found 28709 images belonging to 7 classes.
Found 4394 images belonging to 7 classes.


In [3]:
model = keras.models.load_model('./Pretrained - Copy-MobileNetv2/')
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

 block_3_expand_relu (ReLU)     (None, 56, 56, 144)  0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 57, 57, 144)  0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 28, 28, 144)  1296       ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 28, 28, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_d

 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 14, 14, 384)  24576       ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 14, 14, 384)  1536       ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 14, 14, 384)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 14, 14, 384)  3456       ['block_7_expand_relu[0][0]']    
 nv2D)    

                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 14, 14, 384)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 14, 14, 96)   36864       ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 14, 14, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 14, 14, 576)  55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_

 block_14_depthwise (DepthwiseC  (None, 7, 7, 960)   8640        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 7, 7, 960)   3840        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 7, 7, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 7, 7, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_

In [5]:



# model = keras.applications.MobileNetV3Small(include_top=False, input_shape=(224, 224,3), weights = 'imagenet')


model.trainable = False

base_input = model.layers[0].input

base_output = model.layers[-7].output


final_output = layers.Flatten()(base_output)

final_output=layers.Dropout(0.3)(final_output)


final_output = layers.Dense(128)(final_output)
final_output = layers.Activation('relu')(final_output)

final_output= layers.Dropout(0.5)(final_output)

final_output = layers.Dense(64)(final_output)

final_output = layers.Activation('relu')(final_output)


final_output = layers.Dense(7, activation = 'softmax' )(final_output)

model = keras.Model(inputs=base_input, outputs=final_output)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_pad (ZeroPadding2D)   (None, 225, 225, 3)       0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                             

                                                                 
 conv_pw_8 (Conv2D)          (None, 14, 14, 512)       262144    
                                                                 
 conv_pw_8_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_pw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_dw_9 (DepthwiseConv2D)  (None, 14, 14, 512)      4608      
                                                                 
 conv_dw_9_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_dw_9_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_pw_9

In [ ]:
model.compile(loss = 'categorical_crossentropy', 
              optimizer = keras.optimizers.SGD(learning_rate =0.001), metrics = ['accuracy'], )
model.fit(train_it, batch_size=64 ,epochs =3, verbose=1)

model.evaluate(test_it, batch_size=32, verbose=1)




Epoch 1/3


In [ ]:
model.save('Pretrained')

In [6]:
model = keras.models.load_model('./Pretrained - Copy-MobileNetv2/')
emotionVector = {0: 'Neutral', 1: 'Happy', 2: 'Sad', 3:'Surprise', 4: 'Fearful', 5 : 'Disgusted', 6: 'Angry',    7: 'Contempt'}

In [ ]:
cap = cv2.VideoCapture('./videoplayback.mp4')


while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    frame = cv2.resize(frame, (640, 360))
    if not ret:
        break
    face_detector = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detect faces available on camera
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # take each face available on the camera and Preprocess it
    for (x, y, w, h) in num_faces:
        prediction_arg = 0
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        roi_gray_frame = cv2.resize(roi_gray_frame, (224,224))   
        roi_gray_frame = cv2.merge((roi_gray_frame,roi_gray_frame,roi_gray_frame)) 
        img = np.expand_dims(roi_gray_frame, axis = 0)
        predict_arr=model.predict(img, verbose=0)     
        
        
        prediction_arg= np.argmax(predict_arr)
#         print(prediction_arg)

        cv2.putText(frame, f'{emotionVector[prediction_arg]}', (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    cv2.imshow('Emotion Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()